<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/BingoV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SERVIDOR

In [1]:
%%writefile Servidor.py
import random
import sys
import multiprocessing.shared_memory
import time
import threading
import socket
import os

nuevoNumero = None  # Esta variable se usa para guardar los numertos producidos y para sincronizar al productor y al consumidor
producir = True
maximos_jugadores = 5

# Objeto Condition para sincronizar la producción y el consumo de los numeros
condicion = threading.Condition()

# Para compartir los numeros salientes uso MEMORIA COMPARTIDA
mc = multiprocessing.shared_memory.SharedMemory(name="mi_mem", create=True, size=2)
mc.buf[0]= 0
mc.buf[1]= 0

# Función que produce números
def productor():
  global nuevoNumero
  global producir

  # Creo una lista con todos los posibles numeros, para luego ir sacandolos y asi no se repitan
  numeros = list(range(1, 91))   
  
  while producir:
    with condicion:
      # Esperar si hay un número disponible
      while nuevoNumero is not None:
        condicion.wait()    

      # Tomo un numero aletatorio de la lista y lo remuevo
      nuevoNumero = random.sample(numeros, 1)      
      numeros.remove(nuevoNumero[0])      
      
      # Si la lista esta vacia
      if not numeros:      
        producir = False
            
      # Notificar al consumidor que hay un número disponible
      condicion.notify()

# Función que consume números
def consumidor():
  global nuevoNumero
  global mc  
  global cola_mensajes
  global producir
  global maximos_jugadores
  consumir = True
  bola = 1  
    
  # Para confirmar la recepcion de los datos por parte de los jugadores uso FIFO
  fifo_path = "./fifo"

  # Crear el FIFO si no existe
  if not os.path.exists(fifo_path):
    os.mkfifo(fifo_path, 0o600)

  # Abrir el FIFO en modo lectura
  fifo = os.open(fifo_path, os.O_RDONLY)
  
  while consumir:
    with condicion:
    # Esperar si no hay un número disponible
      while nuevoNumero is None:
        condicion.wait()         
             
      print("Bola numero %i: %i" %(bola, nuevoNumero[0])) 
      bola += 1  

      # Guardo en mc el nuevo valor
      mc.buf[0] = nuevoNumero[0]  
              
      # Espero la confirmacion de recepcion de todos los jugadores
      recibidos = 0     
      
      while recibidos < maximos_jugadores:   
        mensaje = os.read(fifo, 1).decode()
        if mensaje == "":
          pass
        else:   
          recibidos += 1   
          
      # Este sleep es para que la produccion de numeros pueda verse mientras va sucediendo
      time.sleep(0.5)
  
      nuevoNumero = None

      # Si el productor termino tambien termina el consumidor
      if not producir:
        consumir = False  

      # verifico si ya hay un ganador, si es asi finalizo produccion y consumo
      if mc.buf[1] == 1:
        print("\nYa hay un ganador.")  
        consumir = False
        producir = False   
          
      # Notificar al productor que el número ha sido consumido
      condicion.notify()      

  mc.close()   
  os.close(fifo)  
  os.unlink(fifo_path)

#INICIO DEL MAIN
# Esperar a que se conecten los jugadores, uso sockets para notificar su conexion
jugadores_actuales = 0

# Crear un socket
ss = socket.socket()
ss.bind(("127.0.0.1", 2209))

# Escuchar conexiones entrantes
ss.listen(maximos_jugadores)

# Esperar a los jugadores
while jugadores_actuales < maximos_jugadores:
  # Aceptar una conexión entrante
  (cs, dir) = ss.accept()  

  # Incrementar el contador de conexiones
  jugadores_actuales += 1
  print("Se conceto el cliente numero: ", jugadores_actuales)

  # Enviar mensaje al Cliente
  mensaje = str(jugadores_actuales)
  cs.send(mensaje.encode())
  
# Cerrar el socket del servidor
cs.close()
ss.close()

print("\nInicia el bolillero:")

# Crear los hilos
hilo_productor = threading.Thread(target=productor)
hilo_consumidor = threading.Thread(target=consumidor)

# Iniciar los hilos
hilo_productor.start()
hilo_consumidor.start()

# Esperar a que ambos hilos terminen
hilo_productor.join()
hilo_consumidor.join()

# Desvinculo la memoria compartida
mc.unlink()

Writing Servidor.py


JUGADOR

In [2]:
%%writefile Jugador.py
import random
import sys
import multiprocessing.shared_memory
from multiprocessing.resource_tracker import unregister
import socket
import time
import os

def crear_carton():
  # Inicializar el cartón vacío
  carton = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
  
  # Elegir 3 números aleatorios para cada columna del cartón
  for columna in range(9):
    if columna == 0:
      # Primer columna, números del 1 al 9
      numeros = list(range(1, 10))          
    elif columna == 8:       
      # Ultima columna, números del 80 al 90
      numeros = list(range(80, 91))
    else:
      # Resto de columnas, números del (columna * 10) al (columna * 10 + 9)
      numeros = list(range(columna * 10, columna * 10 + 10))
    
    # De los posibles numeros toma solo tres (uno por fila)
    seleccion = random.sample(numeros, 3)  

    # Los coloca en la columna de menor a mayor
    for i in range(3):
      carton[i][columna] = min(seleccion)
      seleccion.remove(min(seleccion)) 

  # Una vez completado el carton, elimina aleatoriamente 4 numeros por fila, para dejar un total de 15 numeros por carton  
  for i in range(3):    
    numeros = list(range(0, 9))           # De las nueve posibles colunas de cada fila
    descarte = random.sample(numeros, 4)  # Toma 4 aleatorias para eliminar su valor (lo pone en 0)
    for num in descarte:
      carton[i][num] = 0                 
  return carton

def imprimir_carton(carton):
  print("Cartón de Bingo:")
  for fila in carton:
    print(fila)

#INICIO DEL MAIN
# conectarse al servidor, uso de sockets
cs = socket.socket()
cs.connect(("127.0.0.1", 2209))

# Recibir mensaje del Servidor (numero de jugador)
mensaje = cs.recv(1024)
jugador = mensaje.decode()
print("Bienvenido, eres el jugador numero ", jugador)
print("")

# Cerrar el socket del cliente
cs.close()

# Crear un cartón de bingo
carton = crear_carton()

# Imprimir el cartón
imprimir_carton(carton)

# Paso valores del carton a una lista
lista_numeros = [i for fila in carton for i in fila]

# Elimino los ceros de la lista para dejar solo los posibles valores de bolas
valor = 0
while valor in lista_numeros:
    lista_numeros.remove(valor)

# Para ver numeros que van saliendo accedo a la memoria compartida
mc = multiprocessing.shared_memory.SharedMemory(name="mi_mem")
unregister(mc._name, "shared_memory")

# Para notificarle al servidor la recepcion de cada valor uso FIFO
fifo_path = "./fifo"

# Crear el FIFO si no existe
if not os.path.exists(fifo_path):
  os.mkfifo(fifo_path, 0o600)

# Abrir el FIFO en modo escritura
fifo = os.open(fifo_path, os.O_WRONLY)

mensaje = str(jugador)

numero = 0  # Lo uso para comparar con el valor leido y saber si es un valor nuevo o uno ya leido anteriromente
bola = 1
consumir = True

print("\nEsperando a los demas jugadores.", end="")

while (mc.buf[0] == 0):
  pass

print("\nEstamos listo!!!\n\nSalio:")

while consumir:
  if mc.buf[1] == 1:
    print("\nFin del juego.")  
    print("Lo sentimos, ya hay un ganador.")      
    consumir = False
  else:
    nuevoNumero = mc.buf[0]  
    if nuevoNumero != numero:    
      numero = nuevoNumero
      print("Bola numero %i: %i" %(bola, numero))
      os.write(fifo, mensaje.encode()) # Notificar recepcion        
      bola += 1       
      if numero in lista_numeros:
        lista_numeros.remove(numero)    
      if not lista_numeros:
        print("\nFelicitaciones, ha ganado !!!")
        mc.buf[1] = 1
        consumir = False   

# Cierro la mc y fifo
mc.close()
os.close(fifo)

Writing Jugador.py


EJECUTAR (inicia al conectarse 5 jugadores)

In [3]:
!nohup python Servidor.py 1>Servidor 2>errorSv &

In [7]:
!nohup python Jugador.py 1>Jugador1 2>/dev/null &

In [6]:
!nohup python Jugador.py 1>Jugador2 2>/dev/null &

In [5]:
!nohup python Jugador.py 1>Jugador3 2>/dev/null &

In [4]:
!nohup python Jugador.py 1>Jugador4 2>/dev/null &

In [8]:
! python Jugador.py

Bienvenido, eres el jugador numero  5

Cartón de Bingo:
[0, 11, 0, 33, 43, 55, 0, 72, 0]
[0, 0, 21, 0, 44, 0, 67, 76, 81]
[7, 0, 0, 35, 45, 0, 68, 78, 0]

Esperando a los demas jugadores.
Estamos listo!!!

Salio:
Bola numero 1: 30
Bola numero 2: 84
Bola numero 3: 85
Bola numero 4: 82
Bola numero 5: 83
Bola numero 6: 2
Bola numero 7: 21
Bola numero 8: 10
Bola numero 9: 55
Bola numero 10: 71
Bola numero 11: 46
Bola numero 12: 36
Bola numero 13: 56
Bola numero 14: 13
Bola numero 15: 22
Bola numero 16: 87
Bola numero 17: 28
Bola numero 18: 45
Bola numero 19: 19
Bola numero 20: 65
Bola numero 21: 64
Bola numero 22: 62
Bola numero 23: 88
Bola numero 24: 52
Bola numero 25: 5
Bola numero 26: 80
Bola numero 27: 49
Bola numero 28: 6
Bola numero 29: 35
Bola numero 30: 40
Bola numero 31: 1
Bola numero 32: 54
Bola numero 33: 16
Bola numero 34: 27
Bola numero 35: 32
Bola numero 36: 29
Bola numero 37: 67
Bola numero 38: 3
Bola numero 39: 33
Bola numero 40: 11
Bola numero 41: 48
Bola numero 42: 25
Bol

EMERGENCIA PARA CUANDO ALGO SE CERRO MAL

In [ ]:
import random
import sys
import multiprocessing.shared_memory
import time

mc = multiprocessing.shared_memory.SharedMemory(name="mi_mem")
mc.close()
mc.unlink()

In [ ]:
import os
fifo_path = "./fifo"
os.unlink(fifo_path)